In [16]:
import numpy as np
import keras

DATA_DIR = '/Users/admin/edu/NLP/practical_NLP_course/'

with open(DATA_DIR+'fiction.lowercased.tokenized.300d') as emb_file:
    lines = emb_file.readlines()

In [9]:
word_dict = {}

In [51]:
with open(DATA_DIR+'fiction.lowercased.tokenized.300d') as emb_file:
    lines = emb_file.readlines()

for line in lines[1:]:
    word, arr = line.split(maxsplit=1)
    word_dict[word] = [float(x) for x in arr.split()]

In [15]:
from conllu import parse

with open(DATA_DIR + "uk_iu-ud-train.conllu", "r") as f:
    data = f.read()

In [54]:
unknown_word = [0.0]*300

Write the data to the disk

In [44]:
trees = parse(data)

with open(DATA_DIR + 'train_vectors.csv', 'a') as output:
    for tree in trees:
        for node in tree:
            head = node["head"]
            child_token = node["form"].lower()
            head_token = (tree[head - 1]["form"] if head > 0 else "root").lower()
            head_vector = word_dict[head_token] if head_token in word_dict else unknown_word
            child_vector = word_dict[child_token] if child_token in word_dict else unknown_word
            if head_vector == unknown_word and child_vector == unknown_word:
                continue
            write_str = ','.join(str(x) for x in head_vector+child_vector+[node["deprel"]])
            output.write(write_str + '\n')

14

In [92]:
import pandas as pd

data = pd.read_csv(DATA_DIR + 'train_vectors.csv', header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,591,592,593,594,595,596,597,598,599,600
0,-0.117512,-0.182687,0.291253,-0.104176,-0.414877,-0.005793,-0.227794,-0.088773,-0.081894,0.073282,...,-0.155744,-0.169492,-0.164372,-0.091375,0.095489,0.076970,0.051420,-0.124548,-0.058590,case
1,-0.035613,-0.002251,0.044372,-0.231303,0.123732,-0.182480,-0.269373,-0.051354,0.019041,0.086354,...,-0.304780,0.310676,-0.011753,-0.084014,0.173566,-0.251616,0.158831,-0.282692,-0.032314,obl
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.166280,0.261123,-0.404187,-0.104432,0.483745,0.915181,0.487084,-0.131554,0.078985,amod
3,-0.117512,-0.182687,0.291253,-0.104176,-0.414877,-0.005793,-0.227794,-0.088773,-0.081894,0.073282,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,nmod
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.027558,-0.043749,0.039582,-0.108463,-0.119397,-0.248160,0.021283,-0.061984,-0.000241,root


In [93]:
from sklearn.feature_extraction.text import CountVectorizer

y_train = data[600]
bow_transformer = CountVectorizer().fit(y_train)
len(bow_transformer.vocabulary_)

44

In [94]:
feature_mapping = bow_transformer.get_feature_names()
y_matrix = bow_transformer.transform(y_train)
y_matr = pd.DataFrame(y_matrix.todense(), columns=feature_mapping)

In [95]:
data.drop([600], axis=1, inplace=True)

In [108]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

model.add(Dense(1024, activation='relu', input_dim=600))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(44, activation='softmax'))

In [109]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [110]:
data.shape, y_matr.shape

((68119, 600), (68119, 44))

In [114]:
model.fit(data, y_matr, epochs=10, batch_size=512)

Epoch 1/10
68119/68119 [==============================] - 9s 134us/step - loss: 1.3143 - acc: 0.6749
Epoch 2/10
68119/68119 [==============================] - 9s 137us/step - loss: 1.2993 - acc: 0.6755 - ETA: 4s - lo
Epoch 3/10
68119/68119 [==============================] - 9s 136us/step - loss: 1.2903 - acc: 0.6775
Epoch 4/10
68119/68119 [==============================] - 9s 138us/step - loss: 1.2946 - acc: 0.6771
Epoch 5/10
68119/68119 [==============================] - 9s 134us/step - loss: 1.2923 - acc: 0.6801
Epoch 6/10
68119/68119 [==============================] - 9s 137us/step - loss: 1.2846 - acc: 0.6808
Epoch 7/10
68119/68119 [==============================] - 9s 137us/step - loss: 1.2864 - acc: 0.6788
Epoch 8/10
68119/68119 [==============================] - 9s 139us/step - loss: 1.2869 - acc: 0.6776
Epoch 9/10
68119/68119 [==============================] - 10s 141us/step - loss: 1.2814 - acc: 0.6825
Epoch 10/10
68119/68119 [==============================] - 10s 140us/step -

## Evaluation

In [115]:
with open(DATA_DIR + "uk_iu-ud-test.conllu", "r") as f:
    data_test = f.read()

In [ ]:
trees_test = parse(data_test)

with open(DATA_DIR + 'test_vectors.csv', 'a') as output:
    for tree in trees_test:
        for node in tree:
            head = node["head"]
            child_token = node["form"].lower()
            head_token = (tree[head - 1]["form"] if head > 0 else "root").lower()
            head_vector = word_dict[head_token] if head_token in word_dict else unknown_word
            child_vector = word_dict[child_token] if child_token in word_dict else unknown_word
            if head_vector == unknown_word and child_vector == unknown_word:
                continue
            write_str = ','.join(str(x) for x in head_vector+child_vector+[node["deprel"]])
            output.write(write_str + '\n')

In [123]:
data_test = pd.read_csv(DATA_DIR + 'test_vectors.csv', header=None)
data_test.head()

,0,1,2,3,4,5,6,7,8,9,...,591,592,593,594,595,596,597,598,599,600
0,-0.250622,0.339085,0.003205,-0.150334,-0.397549,-0.534761,0.148793,-0.116395,-0.061896,0.229367,...,0.120183,0.165805,-0.113048,0.461204,-0.312887,0.252978,-0.056286,-0.530453,-0.052065,nsubj
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.246637,0.224374,-0.647771,-0.775555,-0.177800,-0.549029,0.335552,-0.633770,0.234587,amod
2,-0.229384,0.517341,-0.413346,-0.289054,0.114304,0.517428,-0.394712,0.365179,-0.059949,0.144308,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,nmod
3,-0.250622,0.339085,0.003205,-0.150334,-0.397549,-0.534761,0.148793,-0.116395,-0.061896,0.229367,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,punct
4,-0.250622,0.339085,0.003205,-0.150334,-0.397549,-0.534761,0.148793,-0.116395,-0.061896,0.229367,...,0.284465,0.077792,-0.031626,0.016867,-0.032639,-0.176755,0.047998,-0.119000,-0.033217,expl


In [124]:
y_test = data_test[600]
y_test_matrix = pd.DataFrame(bow_transformer.transform(y_test).todense(), columns=feature_mapping)

In [126]:
data_test.drop([600], axis=1, inplace=True)

In [127]:
score = model.evaluate(data_test, y_test_matrix, batch_size=512)

13400/13400 [==============================] - 1s 59us/step


In [128]:
score

[1.6696751999499193, 0.6335074626509823]